# Flight Data Analysis


In [ ]:
%matplotlib inline
import os
import pymongo
import pandas as pd
db = pymongo.MongoClient(os.environ['MONGO_HOST'])['flirt']
SIM_GROUP = "test-2017-06"


In [ ]:
import datetime
"Notebook evaluated on:" + str(datetime.datetime.now())

In [ ]:
# Spot-check passenger flows
list(db.passengerFlows.find({
    'simGroup': SIM_GROUP,
    'arrivalAirport': 'SEA',
    'departureAirport': 'LAX'
}))

## Total Passenger Counts by Simulation


In [ ]:
results = list(db.passengerFlows.aggregate([{
    '$group': {
        '_id': "$simGroup",
        'total': { '$sum': "$estimatedPassengers" }
    }
}]))
df = pd.DataFrame(results)
df

In [ ]:
results = list(db.schedules.aggregate([{
    "$group": {
        "_id": "$scheduleFileName",
        "minDate": { "$min": "$effectiveDate" }
    }
}]))
df = pd.DataFrame(results)
df

In [ ]:
results = list(db.importedFiles.find())
df = pd.DataFrame(results)
df

## Total domestic and international passenger arrivals by US airport

In [ ]:
USAirports = [airport['_id'] for airport in db.airports.find({"countryName": "United States"})]
results = list(db.passengerFlows.aggregate([{
   "$match": {
       "simGroup": SIM_GROUP,
       "arrivalAirport": {
           "$in": USAirports
       }
   }
}, {
    "$group": {
        "_id": "$arrivalAirport",
        "domestic": {
            "$sum": {
                "$cond": [{"$in": ["$departureAirport", USAirports]}, "$estimatedPassengers", 0]
            }
        },
        "domesticOrigins": {
            "$sum": {
                "$cond": [{"$in": ["$departureAirport", USAirports]}, 1, 0]
            }
        },
        "international": {
            "$sum": {
                "$cond": [{"$in": ["$departureAirport", USAirports]}, 0, "$estimatedPassengers"]
            }
        },
        "internationalOrigins": {
            "$sum": {
                "$cond": [{"$in": ["$departureAirport", USAirports]}, 0, 1]
            }
        },
        "passengerMiles": {
            "$sum": {
                "$multiply": ["$estimatedPassengers", "$averageDistance"]
            }
        }
    }
}]))
inbound_df = pd.DataFrame(results)
inbound_df['combined'] = inbound_df.domestic + inbound_df.international
inbound_df = inbound_df.sort_values(by='combined', ascending=False).set_index('_id')
inbound_df

In [ ]:
import glob
# BTS Market Data Source:
# https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=292
bts_df = pd.read_csv(glob.glob("*_T100_MARKET_ALL_CARRIER.csv")[0])
bts_df = bts_df.query("MONTH == 6")[["PASSENGERS", "ORIGIN", "DEST"]]
bts_df["domestic_origin"] = bts_df.ORIGIN.isin(USAirports)
bts_df["international_origin"] = ~bts_df.domestic_origin
bts_df["domestic_destination"] = bts_df.DEST.isin(USAirports)
bts_df["international_destination"] = ~bts_df.domestic_destination
bts_df["domestic_dest_passengers"] = bts_df.PASSENGERS * bts_df.domestic_destination
bts_df["international_dest_passengers"] = bts_df.PASSENGERS * bts_df.international_destination
bts_df["domestic_passengers"] = bts_df.PASSENGERS * bts_df.domestic_origin
bts_df["international_passengers"] = bts_df.PASSENGERS * bts_df.international_origin


### Compare outbound flight/passengerFlow data to BTS monthly segment/market data
 

In [ ]:
direct_outbound_df = pd.DataFrame(list(
    db.flights.aggregate([{
        '$match': {
            "$and": [{
                'departureDateTime': {
                    '$lte': datetime.datetime(2017, 7, 1)
                }
            }, {
                'departureDateTime': {
                    '$gte': datetime.datetime(2017, 6, 1)
                }
            }],
            'totalSeats': {
                '$gte': 0
            }
        }
    }, {
        '$group': {
            '_id': '$departureAirport',
            'flights': {'$sum' : 1},
            'seats': {'$sum' : '$totalSeats'},
        }
    }
  ])
)).set_index('_id')

In [ ]:
# Check year to year variation in flights
df = pd.DataFrame(list(
    db.flights.aggregate([{
        '$match': {
            "$and": [{
                'departureDateTime': {
                    '$lte': datetime.datetime(2017, 11, 30)
                }
            }, {
                'departureDateTime': {
                    '$gte': datetime.datetime(2016, 6, 1)
                }
            }],
            'totalSeats': {
                '$gte': 0
            }
        }
    }, {
        '$group': {
            '_id': {
                '$concat': ['$departureAirport', '/', '$arrivalAirport', '/', {
                    "$dateToString": {
                        "date": "$departureDateTime",
                        "format": "%Y-%m",
                    }
                }]
            },
            'flights': {'$sum' : 1}
        }
    }])
))
df = pd.concat([df._id.str.split('/', expand=True, n=2), df], axis=1).rename(
    columns={0: 'origin', 1: 'destination', 2:'month'})
df['origin-destination'] = df.origin + '-' + df.destination
df.pivot(index='origin-destination', columns='month', values='flights').corr()

In [ ]:
simulated_outbound_df = pd.DataFrame(list(
 db.passengerFlows.aggregate([
    {
        '$match': {
            "simGroup": SIM_GROUP,
            "departureAirport": {
                "$in": USAirports
            }
        }
    }, {
        '$group': {
            '_id': '$departureAirport',
            'estimatedPassengers': {'$sum' : '$estimatedPassengers'},
            "domesticPassengersSim": {
                "$sum": {
                    "$cond": [{"$in": ["$arrivalAirport", USAirports]}, "$estimatedPassengers", 0]
                }
            },
            "internationalPassengersSim": {
                "$sum": {
                    "$cond": [{"$in": ["$arrivalAirport", USAirports]}, 0, "$estimatedPassengers"]
                }
            },
        }
    }
  ])
)).set_index('_id').sort_values('estimatedPassengers')

In [ ]:
bts_segment_df = pd.read_csv(glob.glob("*_T100_SEGMENT_ALL_CARRIER.csv")[0])\
    .query("MONTH == 6")[["ORIGIN", "DEST", "PASSENGERS", "SEATS", "DEPARTURES_SCHEDULED", "DEPARTURES_PERFORMED"]]
bts_segment_df = bts_segment_df[bts_segment_df.ORIGIN.isin(USAirports)]
result = direct_outbound_df\
    .join(simulated_outbound_df, how="inner")\
    .join(bts_df.groupby("ORIGIN").sum(), rsuffix="_market", how="inner")\
    .join(bts_segment_df.groupby("ORIGIN").sum(), rsuffix="_segment", how="inner")
result = result[~pd.isna(result.flights)].sort_values("flights")
totals = result.sum()
totals

In [ ]:
# Segments per passenger computed from BTS data is lower
# than the value computed from layover probabilities.
# This is most likely due to the BTS market data only including
# a subset of the layovers.
# The BTS market data shows more incoming origins than the direct flight data
# which indicates that it includes at least some multi-leg flights.
# The BTS market data has fewer unique origins that the simulation data.
# That is most likey due to only a sub-set of layovers being included in it.
print "BTS segments per passenger:", totals.PASSENGERS_segment / totals.PASSENGERS

LEG_PROBABILITY_DISTRIBUTION = {
    0: 0.0,
    1: 0.6772732,
    2: 0.2997706,
    3: 0.0211374,
    4: 0.0016254,
    5: 0.0001632,
    6: 0.0000215,
    7: 0.0000072,
    8: 0.0000012,
    9: 0.0000002,
    10: 0.0000001
}
lpd_segments_per_passenger = sum(legs * value for legs, value in LEG_PROBABILITY_DISTRIBUTION.items())
print "Leg dist segments per passenger:", lpd_segments_per_passenger

In [ ]:
print "Simulated passengers / BTS market passengers =", totals.estimatedPassengers / totals.PASSENGERS
print "Simulated passengers / BTS segment seats =", totals.estimatedPassengers / totals.SEATS
print "Direct flight seats / BTS segment seats =", totals.seats / totals.SEATS

In [ ]:
# Flights extracted from schedules are between the scheduled and performed departures in BTS data.
print "Scheduled departures / BTS departures =", totals.flights / totals.DEPARTURES_PERFORMED
print "Scheduled departures / BTS scheduled departures =", totals.flights / totals.DEPARTURES_SCHEDULED

In [ ]:
# Compute total segment-seats from estimated total passengers.
# This should be close to the number of seats in the flight scheulde data.
est_total_seats_used = totals.estimatedPassengers * lpd_segments_per_passenger
load_factor = totals.PASSENGERS_segment / totals.SEATS
est_total_seats = est_total_seats_used / load_factor
print "Estimated total seats / schdeuled seats =", est_total_seats / totals.seats

In [ ]:
# Find airports with few estimated passengers an many actual passengers because
# they may have been simulated incorrectly.
result['ratio'] = result.estimatedPassengers / result.PASSENGERS
result.query("ratio < .5").sort_values('PASSENGERS')

### Compare Incoming passenger flows with BTS data

In [ ]:
direct_incoming_df = pd.DataFrame(list(
    db.flights.aggregate([
       {
           '$match': {
               "$and": [{
                   'departureDateTime': {
                       '$lte': datetime.datetime(2017, 7, 1)
                   }
               }, {
                   'departureDateTime': {
                       '$gte': datetime.datetime(2017, 6, 1)
                   }
               }],
               'totalSeats': {
                   '$gte': 0
               }
           }
       }, {
           '$group': {
               '_id': {
                   '$concat': ['$departureAirport', '-', '$arrivalAirport']
               },
               'arrivalAirport': { '$first': '$arrivalAirport' },
               'departureAirport': { '$first': '$departureAirport' },
               'flights': {'$sum' : 1},
               'seats': {'$sum' : '$totalSeats'},
           }
       }, {
           '$group': {
               '_id': '$arrivalAirport',
               "directDomesticOrigins": {
                   "$sum": {
                       "$cond": [{"$in": ["$departureAirport", USAirports]}, 1, 0]
                   }
               },
               "directInternationalOrigins": {
                   "$sum": {
                       "$cond": [{"$in": ["$departureAirport", USAirports]}, 0, 1]
                   }
               },
               'flights': {'$sum' : '$flights'},
               'seats': {'$sum' : '$seats'},
           }
       }
    ])
)).set_index('_id')

In [ ]:
inbound_df.join(bts_df.groupby("DEST").sum()).rename(columns={
    "PASSENGERS": "Total Passengers (actual)",
    "domestic_passengers": "Domestic Passengers (actual)",
    "international_passengers": "International Passengers (actual)",
    "combined": "Total Passengers (simulated)",
    "domestic": "Domestic Passengers (simulated)",
    "international": "International Passengers (simulated)",
    "domestic_origin": "Number of Domestic Origins (actual)",
    "international_origin": "Number of International Origins (actual)"
}).join(direct_incoming_df)


In [ ]:
seatac_dest_oct_2016_rankings = [
    "LAX",
    "SFO",
    "ANC",
    "DEN",
    "LAS",
    "PHX",
    "ORD",
    "PDX",
    "DFW",
    "GEG"
]
results = list(db.passengerFlows.aggregate([{
   "$match": {
       "departureAirport": "SEA",
       "simGroup": {
           "$in": [
               "fmd-2017-03",
               "fmd-2017-04",
               "fmd-2017-05",
               "fmd-2017-06",
               "fmd-2017-07",
               "fmd-2017-08",
               "fmd-2017-09"
           ]
       },
       "arrivalAirport": {
           "$in": USAirports
       }
   }
}, {
    "$group": {
        "_id": "$arrivalAirport",
        "total": { "$sum": "$estimatedPassengers" }
    }
}]))
pd.DataFrame(results).sort_values(by="total", ascending=False)[:15]


In [ ]:
# Num flights out vs in
out_counts_map = {}
departure_airports = set()
arrival_airprots = set()
for doc in db.flights.aggregate([{
    "$group": {
        "_id": "$departureAirport",
        "total": { "$sum": 1 }
    }
}]):
    departure_airports.add(doc['_id'])
    out_counts_map[doc['_id']] = doc['total']
ratios = []
for doc in db.flights.aggregate([{
    "$group": {
        "_id": "$arrivalAirport",
        "total": { "$sum": 1 }
    }
}]):
    arrival_airprots.add(doc['_id'])
    out_count = out_counts_map.get(doc['_id'], None)
    if out_count:
        ratios.append(dict(
            _id=doc['_id'],
            ratio=doc['total'] / out_count))
print("Inbound only airports", arrival_airprots - departure_airports)
print("Outbound only airports", departure_airports - arrival_airprots)
pd.DataFrame(ratios).sort_values(by="ratio")

In [ ]:
results = db.flights.aggregate([{
    "$group": {
        "_id": {
            "$concat": [{
                "$dateToString": {
                    "date": "$departureDateTime",
                    "format": "%Y-%m-%d",
                }
            }, "_", "$scheduleFileName"]
        },
        "totalFlights": { "$sum": 1 },
        "totalSeats": { "$sum": "$totalSeats" }
    }
}])
df = pd.DataFrame(list(results))
parsed_df = pd.concat([df._id.str.split('_', expand=True, n=1), df], axis=1).rename(
    columns={0: 'date', 1: 'file'})
parsed_df.date = pd.to_datetime(parsed_df.date)
parsed_df[["date", "totalFlights", "totalSeats", "file"]]
parsed_df


In [ ]:
parsed_df[parsed_df.date < pd.Timestamp("2019-1-1")].plot(
    x="date",
    y="totalSeats",
    kind="line",
    title="Daily Seats",
    figsize=(12, 6))

In [ ]:
parsed_df.pivot(index='date', columns='file', values='totalSeats').reset_index().plot(
    x="date",
    kind="line",
    title="Daily Flights",
    figsize=(12, 6))
